## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,68.74,94,94,8.79,light rain
1,1,Kultuk,RU,51.7208,103.6956,36.77,93,100,6.91,overcast clouds
2,2,Mataura,NZ,-46.1927,168.8643,50.49,89,99,3.31,overcast clouds
3,3,Hobart,AU,-42.8794,147.3294,55.27,97,75,1.01,light intensity drizzle
4,4,Provideniya,RU,64.3833,-173.3000,20.98,80,6,6.64,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 66
What is the maximum temperature you would like for your trip? 67


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
18,18,Carnarvon,AU,-24.8667,113.6333,66.27,63,0,17.27,clear sky
45,45,La Rioja,ES,42.2500,-2.5000,66.04,35,64,6.58,broken clouds
164,164,Luan,CH,46.3592,6.9759,66.56,61,47,1.99,scattered clouds
453,453,Maltahohe,NaN,-24.8333,16.9833,66.83,14,0,7.72,clear sky
538,538,Itirapina,BR,-22.2528,-47.8228,66.13,65,98,5.57,overcast clouds
562,562,Yataity,PY,-26.3667,-56.0667,66.90,99,100,2.59,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        6
City           6
Country        5
Lat            6
Lng            6
Max Temp       6
Humidity       6
Cloudiness     6
Wind Speed     6
Description    6
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
18,18,Carnarvon,AU,-24.8667,113.6333,66.27,63,0,17.27,clear sky
45,45,La Rioja,ES,42.2500,-2.5000,66.04,35,64,6.58,broken clouds
164,164,Luan,CH,46.3592,6.9759,66.56,61,47,1.99,scattered clouds
538,538,Itirapina,BR,-22.2528,-47.8228,66.13,65,98,5.57,overcast clouds
562,562,Yataity,PY,-26.3667,-56.0667,66.90,99,100,2.59,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
18,Carnarvon,AU,66.27,clear sky,-24.8667,113.6333,
45,La Rioja,ES,66.04,broken clouds,42.2500,-2.5000,
164,Luan,CH,66.56,scattered clouds,46.3592,6.9759,
453,Maltahohe,NaN,66.83,clear sky,-24.8333,16.9833,
538,Itirapina,BR,66.13,overcast clouds,-22.2528,-47.8228,
562,Yataity,PY,66.90,overcast clouds,-26.3667,-56.0667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel = requests.get(base_url, params=params)
    
    hotel = hotel.json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset=['Hotel Name'])

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
18,Carnarvon,AU,66.27,clear sky,-24.8667,113.6333,Hospitality Carnarvon
45,La Rioja,ES,66.04,broken clouds,42.2500,-2.5000,Casa Concejos
164,Luan,CH,66.56,scattered clouds,46.3592,6.9759,Hotel Appartement Castel Club Leysin Parc
453,Maltahohe,NaN,66.83,clear sky,-24.8333,16.9833,Maltahöhe Hotel
538,Itirapina,BR,66.13,overcast clouds,-22.2528,-47.8228,Pousada Paraiso das Aguas
562,Yataity,PY,66.90,overcast clouds,-26.3667,-56.0667,Cristhian Ortigoza


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))